In [1]:
import pandas as pd

# Load the dataset
# Replace with the actual path to your CSV file
df = pd.read_csv('/content/cleaned_dataset_combined (4).csv')

# Display the first few rows to verify
print("Original DataFrame:")
print(df.head())
print(df.info())

Original DataFrame:
   hate_speech  offensive_language  neither  class  \
0            0                   0        3      1   
1            0                   3        0      0   
2            0                   3        0      0   
3            0                   2        1      0   
4            0                   6        0      0   

                                               tweet  \
0  rt as a woman you should not complain about cl...   
1  rt boy dats coldtyga dwn bad for cufin dat hoe...   
2  rt dawg rt you ever fuck a bitch and she start...   
3                            rt she lok like a trany   
4  rt the shit you hear about me might be true or...   

                                        tweet_tokens  \
0  ['rt', 'woman', 'complain', 'cleaning', 'house...   
1  ['rt', 'boy', 'dat', 'coldtyga', 'dwn', 'bad',...   
2  ['rt', 'dawg', 'rt', 'ever', 'fuck', 'bitch', ...   
3                     ['rt', 'lok', 'like', 'trany']   
4  ['rt', 'shit', 'hear', 'might', 'tr

In [2]:
import ast

# Convert string representation of list to actual list
df['tweet_tokens'] = df['tweet_tokens'].apply(ast.literal_eval)

# Display the first few rows to verify
print("\nDataFrame after converting tweet_tokens to lists:")
print(df.head())
print(df.info())


DataFrame after converting tweet_tokens to lists:
   hate_speech  offensive_language  neither  class  \
0            0                   0        3      1   
1            0                   3        0      0   
2            0                   3        0      0   
3            0                   2        1      0   
4            0                   6        0      0   

                                               tweet  \
0  rt as a woman you should not complain about cl...   
1  rt boy dats coldtyga dwn bad for cufin dat hoe...   
2  rt dawg rt you ever fuck a bitch and she start...   
3                            rt she lok like a trany   
4  rt the shit you hear about me might be true or...   

                                        tweet_tokens  \
0  [rt, woman, complain, cleaning, house, man, al...   
1  [rt, boy, dat, coldtyga, dwn, bad, cufin, dat,...   
2  [rt, dawg, rt, ever, fuck, bitch, start, cry, ...   
3                             [rt, lok, like, trany]   
4  [rt,

In [3]:
from gensim.models import Word2Vec

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=df['tweet_tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Get the vector size
vector_size = word2vec_model.wv.vector_size


In [4]:
import numpy as np

def document_vector(doc):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    # If the document is empty, return a zero vector
    if not doc:
        return np.zeros(vector_size)
    # Calculate the mean vector
    return np.mean(word2vec_model.wv[doc], axis=0)

# Apply the function to transform tweet_tokens
X_vectors = df['tweet_tokens'].apply(document_vector).tolist()
X_vectors = np.array(X_vectors)


In [5]:


# Define the target variable
y = df['class']


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_vectors, y, test_size=0.2, random_state=42)

# Create a pipeline with StandardScaler and RandomForestClassifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize the features
    ('classifier', RandomForestClassifier())
])

# Train the model
pipeline.fit(X_train, y_train)

# Evaluate the model
accuracy = pipeline.score(X_test, y_test)
print(f'\nAccuracy: {accuracy}')



Accuracy: 0.9027048849414614
